# 抓Mobile 01 （以Iphone板為例）

In [ ]:
"""   目 錄 頁 規 則   """
# 進入主頁   ： https://www.mobile01.com/topiclist.php?f=383
# 第2頁     ： https://www.mobile01.com/topiclist.php?f=383&p=2
# 第3頁     ： https://www.mobile01.com/topiclist.php?f=383&p=3
# 再回到第1頁： https://www.mobile01.com/topiclist.php?f=383&p=1

In [ ]:
# 建立存放位置
!mkdir -p /tmp/mobile01_data

In [ ]:
import re
import time

import requests

DATA_PATH = '/tmp/mobile01_data'
URL_TEMPLATE = "https://www.mobile01.com/topiclist.php?f=383&p={}"
HOST = "https://www.mobile01.com"

In [ ]:
"""   由目錄頁的html file中觀察導向內容頁的url規則 (inspect後的 edit attribute 來copy url)   """
# topicdetail.php?f=383&t=5363464
# topicdetail.php?f=383&t=5348939
# topicdetail.php?f=383&t=5307721

In [ ]:
# 測試regular expression的結果   註：『 \d 』 等同於傳統的[0-9], python 3之後可以簡寫成 \d
url = "https://www.mobile01.com/topiclist.php?f=383&p=1"
resp = requests.get(url)
links = re.findall('href="(topicdetail.php\?f=383&t=\d+)"', resp.text)
detail_page_links = ["%s/%s"%(HOST, link) for link in links]
print(links)

In [ ]:
# 目錄頁為url，丟進去get_list_page() 後取得內文頁的連結們

def get_list_page(url):
    resp = requests.get(url)
    links = re.findall('<a href="(topicdetail.php\?f=383&t=\d+)">', resp.text)
    detail_page_links = ["%s/%s"%(HOST, link) for link in links]
    return detail_page_links

In [ ]:
def dump_page(url):
    filename = "_".join(re.findall('f=(\d+)&t=(\d+)&p=(\d+)', url)[0]) + '.html'
    resp = requests.get(url)

    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return resp

In [ ]:
""" 每個內文頁是個個別主頁， 若回文數超過一頁會變成第2頁, 第3頁... 去inspect如何去連結的關聯  """
# 首頁的url為 → https://www.mobile01.com/topicdetail.php?f=383&t=5313803
#                             <a href="topicdetail.php?f=383&amp;t=5313803&amp;p=2">2</a>
#                             <a href="topicdetail.php?f=383&amp;t=5313803&amp;p=3">3</a>
#                             <a href="topicdetail.php?f=383&amp;t=5313803&amp;p=4">4</a>

""" 若先到第2頁，再回頭連第1頁的網址是： """
#                             <a href="topicdetail.php?f=383&amp;t=5313803&amp;p=1">1</a>

In [ ]:
""" 機車的一點是Mobile01的網頁即使該主題只有1頁， 可是輸入p=2, p=3...系統還是可以執行，只是同樣顯示第1頁的資訊，
    因此用爬蟲爬的話就會無限loop抓取同樣的頁面，存檔成p=2, p=3...直到塞爆硬碟為止！ """
# canonical address + while True來解決

In [ ]:
""" 若網站有限制抓網的速度的話(執行中但沒有新產出時)， 引用time.sleep(秒速)來降低速度  """

In [ ]:
for page in range(1,2):
    url = URL_TEMPLATE.format(page)
    for link in get_list_page(url):
        link_pg = 1
        while True:
            """
            內文頁換頁邏輯
            """
            link_url = link + "&p=%s"%link_pg
            print(link_url)
            resp = dump_page(link_url)
            time.sleep(0.2)
            
            """
            用canonical address來做判斷是否爬到最後一頁
            """
            canonical = re.findall('<link rel="canonical" href="(.+)" />', resp.text)[0]
            if link_pg == 1: #第一頁的特別處理，拿掉p=1
                link_url = link_url[:-4]
            if canonical != link_url:
                break
            link_pg += 1